<a href="https://colab.research.google.com/github/Gayathri288/231801039_PODS/blob/main/Ex_4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle pandas scikit-learn spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
from google.colab import files
uploaded = files.upload()


Saving 1.png to 1.png


In [10]:
import os

os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [11]:
!kaggle datasets download -d snap/amazon-fine-food-reviews
!unzip -q amazon-fine-food-reviews.zip


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open amazon-fine-food-reviews.zip, amazon-fine-food-reviews.zip.zip or amazon-fine-food-reviews.zip.ZIP.


In [13]:
import pandas as pd
df = pd.read_csv("Reviews.csv")
df = df[df['Text'].notnull()]
df = df.head(1000)
df = df[['Id', 'Score', 'Summary', 'Text']]
df.head()


,Id,Score,Summary,Text
0,1,5,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,4,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,2,Cough Medicine,If you are looking for the secret ingredient i...
4,5,5,Great taffy,Great taffy at a great price. There was a wid...


In [14]:
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [
        token.lemma_ for token in doc
        if token.is_alpha and not token.is_stop
    ]
    return " ".join(tokens)

df['cleaned_text'] = df['Text'].apply(preprocess_text)
df[['Text', 'cleaned_text']].head()


,Text,cleaned_text
0,I have bought several of the Vitality canned d...,buy vitality can dog food product find good qu...
1,Product arrived labeled as Jumbo Salted Peanut...,product arrived label jumbo salt peanut peanut...
2,This is a confection that has been around a fe...,confection century light pillowy citrus gelati...
3,If you are looking for the secret ingredient i...,look secret ingredient robitussin believe find...
4,Great taffy at a great price. There was a wid...,great taffy great price wide assortment yummy ...


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['cleaned_text'])


In [16]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def preprocess_query(query):
    return preprocess_text(query)

def vectorize_query(query):
    cleaned_query = preprocess_query(query)
    return vectorizer.transform([cleaned_query])

def find_similar_reviews(query, top_k=5):
    query_vec = vectorize_query(query)
    similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    top_indices = np.argsort(similarities)[::-1][:top_k]
    return df.iloc[top_indices][['Id', 'Score', 'Summary', 'Text']]


In [17]:
query = "This product tastes amazing and I love it"
results = find_similar_reviews(query, top_k=5)
results


,Id,Score,Summary,Text
181,182,5,Marley's Mellow Mood Lite - Half Tea Half Lemo...,"This is an great product. The taste is great, ..."
346,347,4,Love the Fresh Food Feeder!!,This product is amazing. I know a few people w...
495,496,5,amazing chips,i rarely eat chips but i saw these and tried t...
105,106,5,The best,I like Creme Brulee. I loved that these were s...
344,345,3,Wonderful idea-difficult to clean.,"I love, love, love the idea of this product an..."
